# Integrated Mining Analysis: Structural Features + Mineralization

## Objectives
- **Combine outputs** from `mining-aster.ipynb` and `mining-palsar.ipynb`
- **Identify priority exploration targets** by integrating:
  - Structural features (lineaments, faults, fractures)
  - Alteration zones (clay, iron, carbonates)
  - Mineralization probability
- **Create exploration target maps** with confidence scoring
- **Generate priority target classifications** for field investigation

## Data Sources
- **Structural Features:** From `mining-palsar.ipynb`
  - Lineament maps (DEM + PALSAR)
  - Structural feature classification (faults, fractures, complexity zones)
  - Lineament density
  - Lineament orientation
  
- **Mineralization/Alteration:** From `mining-aster.ipynb`
  - Clay, Iron, Carbonate indices
  - Mineralization probability maps
  - Alteration composite maps

## Study Area
- Coordinates (WGS1984):
  - Point 1: 04°46'0.00"N, 29°34'30.00"E
  - Point 2: 04°46'0.00"N, 29°16'45.00"E
  - Point 3: 05°05'30.00"N, 29°16'45.00"E
  - Point 4: 05°05'30.00"N, 29°34'30.00"E
- Buffer: 20m
- Analysis Period: 2023-2024

## Workflow
1. **Prerequisites:** Run `mining-aster.ipynb` and `mining-palsar.ipynb` first
2. **Load outputs:** Import exported GeoTIFFs or Earth Engine assets
3. **Integrated analysis:** Combine structural and mineralization data
4. **Target identification:** Generate priority exploration targets
5. **Export results:** Save integrated target maps


In [ ]:
# Install required packages
!pip install geemap earthengine-api numpy rasterio


In [ ]:
# Import required libraries
import geemap
import ee
import numpy as np

# Initialize Earth Engine
# Replace 'your-project-id' with your Google Cloud Project ID
PROJECT_ID = 'your-project-id'  # UPDATE THIS!

try:
    geemap.ee_initialize(project=PROJECT_ID)
    print(f"Earth Engine initialized with project: {PROJECT_ID}")
except Exception as e:
    print(f"Initialization error: {e}")
    print("Please authenticate: geemap.ee_initialize()")

print("Libraries imported and geemap initialized successfully!")


In [ ]:
# Define study area (same as other notebooks)
# Coordinates in WGS1984
point1 = [29.575, 4.766667]  # 29°34'30.00"E, 04°46'0.00"N
point2 = [29.279167, 4.766667]  # 29°16'45.00"E, 04°46'0.00"N
point3 = [29.279167, 5.091667]  # 29°16'45.00"E, 05°05'30.00"N
point4 = [29.575, 5.091667]  # 29°34'30.00"E, 05°05'30.00"N

# Create polygon
study_area = ee.Geometry.Polygon([[
    point1, point2, point3, point4, point1
]])

# Apply 20m buffer
study_area_buffered = study_area.buffer(20)

print("Study area created with 20m buffer")
print(f"Area: {study_area_buffered.area().getInfo() / 1e6:.2f} km²")

# Create map centered on study area
Map = geemap.Map(center=[4.929, 29.427], zoom=13)
Map.addLayer(study_area_buffered, {'color': 'blue'}, 'Study Area (20m buffer)')
Map


## Load Outputs from Previous Analyses

This notebook automatically detects and uses outputs from `mining-aster.ipynb` and `mining-palsar.ipynb` if they're in the same kernel session.

**How it works:**
1. **First checks** for existing variables from other notebooks (if run in same kernel)
2. **Uses them directly** - no export/import needed!
3. **Falls back** to recalculating only if variables are missing

**To use existing outputs:**
- Run `mining-aster.ipynb` (all cells) → creates `mineralisation_2023_2024`
- Run `mining-palsar.ipynb` (all cells) → creates `structural_features_classified`, `lineament_density_2023_2024`
- Run this notebook → automatically detects and uses them!

**Alternative:** If outputs aren't available, the notebook will recalculate simplified versions.


In [ ]:
# Load Structural Features from PALSAR Analysis
# First checks for existing variables from mining-palsar.ipynb

print("=" * 60)
print("LOADING STRUCTURAL FEATURES (PALSAR/SRTM)")
print("=" * 60)
print("Checking for existing outputs from mining-palsar.ipynb...")

# Check for existing structural features (from mining-palsar.ipynb)
structural_features_loaded = False
lineament_density_loaded = False
structural_features = None
lineament_density = None

if 'structural_features_classified' in globals() and structural_features_classified is not None:
    structural_features = structural_features_classified
    print("✓ Found: structural_features_classified (from mining-palsar.ipynb)")
    print("  Using existing variable - no recalculation needed!")
    structural_features_loaded = True
else:
    print("⚠️  structural_features_classified not found in current session")

if 'lineament_density_2023_2024' in globals() and lineament_density_2023_2024 is not None:
    lineament_density = lineament_density_2023_2024
    print("✓ Found: lineament_density_2023_2024 (from mining-palsar.ipynb)")
    print("  Using existing variable - no recalculation needed!")
    lineament_density_loaded = True
else:
    print("⚠️  lineament_density_2023_2024 not found in current session")

if not structural_features_loaded or not lineament_density_loaded:
    print("\n⚠️  Some structural features missing - will use simplified calculation")
    print("   For best results, run mining-palsar.ipynb first (all cells)")
    print("   Then run this notebook in the same kernel session")
    
    # Load SRTM DEM for simplified calculation (if needed)
    if 'srtm' not in globals():
        srtm = ee.Image("USGS/SRTMGL1_003").clip(study_area_buffered)
        print("✓ SRTM DEM loaded for simplified calculation")
else:
    print("\n✓ All structural features found! Using outputs directly from mining-palsar.ipynb")

print("=" * 60)


In [ ]:
# Load Mineralization/Alteration from ASTER Analysis
# First checks for existing variables from mining-aster.ipynb

print("=" * 60)
print("LOADING MINERALIZATION/ALTERATION (ASTER)")
print("=" * 60)
print("Checking for existing outputs from mining-aster.ipynb...")

mineralization_loaded = False
mineralization_probability = None

# Check for existing mineralization map (from mining-aster.ipynb)
if 'mineralisation_2023_2024' in globals():
    mineralization_probability = mineralisation_2023_2024.select('Mineralisation_Probability')
    print("✓ Found: mineralisation_2023_2024 (from mining-aster.ipynb)")
    print("  Using existing variable - no recalculation needed!")
    mineralization_loaded = True
elif 'aster_2023_2024_clay' in globals() and 'aster_2023_2024_iron' in globals() and 'aster_2023_2024_carbonate' in globals():
    # Recreate mineralization from alteration indices if available
    print("✓ Found: alteration indices (from mining-aster.ipynb)")
    clay_prob = aster_2023_2024_clay.select('Clay_Index').subtract(0.8).multiply(10).clamp(0, 1)
    iron_prob = aster_2023_2024_iron.select('Iron_Index').subtract(0.8).multiply(5).clamp(0, 1)
    carbonate_prob = aster_2023_2024_carbonate.select('Carbonate_Index').subtract(0.9).multiply(10).clamp(0, 1)
    mineralization_probability = (
        clay_prob.multiply(0.4)
        .add(iron_prob.multiply(0.3))
        .add(carbonate_prob.multiply(0.3))
    ).rename('Mineralisation_Probability')
    print("✓ Created: mineralization from existing alteration indices")
    mineralization_loaded = True
else:
    print("⚠️  ASTER outputs not found - will recalculate")
    
    # Recalculate if not found
    aster_collection = ee.ImageCollection("ASTER/AST_L1T_003").filterDate('2023-01-01', '2024-12-31').filterBounds(study_area_buffered)
    aster_count = aster_collection.size().getInfo()
    print(f"Number of ASTER images (2023-2024): {aster_count}")
    
    if aster_count > 0:
        aster_2023_2024 = aster_collection.median().clip(study_area_buffered)
        print("✓ ASTER composite created")
        
        # Calculate alteration indices
        clay_index = aster_2023_2024.expression(
            '(B7 / B5) * (B4 / B6)',
            {'B4': aster_2023_2024.select('B04'),
             'B5': aster_2023_2024.select('B05'),
             'B6': aster_2023_2024.select('B06'),
             'B7': aster_2023_2024.select('B07')}
        ).rename('Clay_Index')
        
        iron_index = aster_2023_2024.expression(
            'B4 / B2',
            {'B2': aster_2023_2024.select('B02'),
             'B4': aster_2023_2024.select('B04')}
        ).rename('Iron_Index')
        
        carbonate_index = aster_2023_2024.expression(
            'B13 / B14',
            {'B13': aster_2023_2024.select('B13'),
             'B14': aster_2023_2024.select('B14')}
        ).rename('Carbonate_Index')
        
        # Create mineralization probability map
        clay_prob = clay_index.subtract(0.8).multiply(10).clamp(0, 1)
        iron_prob = iron_index.subtract(0.8).multiply(5).clamp(0, 1)
        carbonate_prob = carbonate_index.subtract(0.9).multiply(10).clamp(0, 1)
        
        mineralization_probability = (
            clay_prob.multiply(0.4)
            .add(iron_prob.multiply(0.3))
            .add(carbonate_prob.multiply(0.3))
        ).rename('Mineralisation_Probability')
        
        print("✓ Alteration indices calculated")
        print("✓ Mineralization probability map created")
        mineralization_loaded = True
    else:
        print("⚠️  No ASTER data available for 2023-2024")
        print("   Please run mining-aster.ipynb first to generate outputs")
        mineralization_loaded = False

if mineralization_loaded:
    print("\n✓ Mineralization data ready!")
else:
    print("\n⚠️  Mineralization data not available")

print("=" * 60)


In [ ]:
# Load Mineralization/Alteration from ASTER Analysis
# This replicates key outputs from mining-aster.ipynb

print("=" * 60)
print("LOADING MINERALIZATION/ALTERATION (ASTER)")
print("=" * 60)

# Load ASTER data for 2023-2024
aster_collection = ee.ImageCollection("ASTER/AST_L1T_003").filterDate('2023-01-01', '2024-12-31').filterBounds(study_area_buffered)
aster_count = aster_collection.size().getInfo()
print(f"Number of ASTER images (2023-2024): {aster_count}")

if aster_count > 0:
    aster_2023_2024 = aster_collection.median().clip(study_area_buffered)
    print("✓ ASTER composite created")
    
    # Calculate alteration indices (simplified versions)
    # Clay Index: (B7/B5) * (B4/B6)
    clay_index = aster_2023_2024.expression(
        '(B7 / B5) * (B4 / B6)',
        {'B4': aster_2023_2024.select('B04'),
         'B5': aster_2023_2024.select('B05'),
         'B6': aster_2023_2024.select('B06'),
         'B7': aster_2023_2024.select('B07')}
    ).rename('Clay_Index')
    
    # Iron Index: B4/B2
    iron_index = aster_2023_2024.expression(
        'B4 / B2',
        {'B2': aster_2023_2024.select('B02'),
         'B4': aster_2023_2024.select('B04')}
    ).rename('Iron_Index')
    
    # Carbonate Index: B13/B14
    carbonate_index = aster_2023_2024.expression(
        'B13 / B14',
        {'B13': aster_2023_2024.select('B13'),
         'B14': aster_2023_2024.select('B14')}
    ).rename('Carbonate_Index')
    
    # Create mineralization probability map
    clay_prob = clay_index.subtract(0.8).multiply(10).clamp(0, 1)
    iron_prob = iron_index.subtract(0.8).multiply(5).clamp(0, 1)
    carbonate_prob = carbonate_index.subtract(0.9).multiply(10).clamp(0, 1)
    
    mineralization_probability = (
        clay_prob.multiply(0.4)
        .add(iron_prob.multiply(0.3))
        .add(carbonate_prob.multiply(0.3))
    ).rename('Mineralisation_Probability')
    
    print("✓ Alteration indices calculated")
    print("✓ Mineralization probability map created")
    mineralization_loaded = True
else:
    print("⚠️  No ASTER data available for 2023-2024")
    print("   Please run mining-aster.ipynb first to generate outputs")
    mineralization_loaded = False

print("=" * 60)


## Interpretation Guide

### Priority Target Identification

**High Priority Targets (Red):**
- **Score >0.7** - Best exploration targets
- High structural complexity (>5/km² lineament density)
- High mineralization probability (>0.6)
- Strong structural control
- **Action:** Priority for field investigation and sampling

**Medium Priority Targets (Orange):**
- **Score 0.5-0.7** - Good exploration targets
- Moderate structural complexity (3-5/km²)
- Moderate mineralization (0.4-0.6)
- **Action:** Secondary targets for follow-up

**Low Priority Targets (Yellow):**
- **Score 0.3-0.5** - Potential targets
- Lower scores but still favorable
- **Action:** Lower priority, may be worth investigating if other targets are exhausted

### Specific Target Types

**Priority Targets (Magenta):**
- Highest density zones (>5/km²) with highest mineralization (>0.6)
- Top priority for field investigation
- Likely zones of active mineralization

**Structural Control Zones (Cyan):**
- Faults directly intersecting mineralization zones
- Critical for understanding ore controls
- Indicates structural pathways for mineralizing fluids

**Favorable Zones (Green):**
- Medium-high density (3-5/km²) with mineralization (>0.5)
- Good potential but lower priority than magenta zones


In [ ]:
# INTEGRATED ANALYSIS: Combine Structural Features + Mineralization
# This is the core integration analysis

print("=" * 60)
print("INTEGRATED STRUCTURAL-MINERALIZATION ANALYSIS")
print("=" * 60)
print("Using outputs directly from other notebooks (if available)")
print("=" * 60)

def create_integrated_targets(structural_features, lineament_density, mineralization_prob):
    """
    Create integrated exploration target map combining:
    - Structural features (faults, fractures, complexity zones)
    - Lineament density
    - Mineralization probability
    """
    
    # 1. PRIORITY TARGETS: High structural complexity + High mineralization
    high_density = lineament_density.gt(5)
    high_mineralization = mineralization_prob.gt(0.6)
    priority_targets = high_density.And(high_mineralization).rename('Priority_Targets')
    
    # 2. STRUCTURAL CONTROL ZONES: Faults intersecting mineralization
    faults = structural_features.select('Faults')
    medium_mineralization = mineralization_prob.gt(0.4)
    structural_control = faults.And(medium_mineralization).rename('Structural_Control')
    
    # 3. FAVORABLE ZONES: Alteration along lineaments
    favorable_zones = lineament_density.gt(3).And(mineralization_prob.gt(0.5)).rename('Favorable_Zones')
    
    # 4. EXPLORATION TARGET SCORE: Weighted combination
    density_norm = lineament_density.unitScale(0, 10)  # Normalize 0-10/km² to 0-1
    mineral_norm = mineralization_prob  # Already 0-1
    
    # Structural complexity score
    complexity_score = structural_features.select('Structural_Complexity').add(
        structural_features.select('Faults').multiply(0.8)
    ).add(
        structural_features.select('Shear_Zones').multiply(0.5)
    ).unitScale(0, 2.3)
    
    # Combined exploration score (weighted)
    exploration_score = (
        density_norm.multiply(0.3)  # 30% structure density
        .add(mineral_norm.multiply(0.4))  # 40% mineralization
        .add(complexity_score.multiply(0.3))  # 30% structural complexity
    ).rename('Exploration_Score')
    
    # Classify into target categories
    high_targets = exploration_score.gt(0.7).rename('High_Priority')
    medium_targets = exploration_score.gt(0.5).And(exploration_score.lte(0.7)).rename('Medium_Priority')
    low_targets = exploration_score.gt(0.3).And(exploration_score.lte(0.5)).rename('Low_Priority')
    
    # Combine all outputs
    integrated = ee.Image.cat([
        priority_targets,
        structural_control,
        favorable_zones,
        exploration_score,
        high_targets,
        medium_targets,
        low_targets
    ])
    
    return integrated

# Check if we have all required inputs
if mineralization_loaded and structural_features_loaded and lineament_density_loaded:
    print("✓ All required inputs found!")
    print("  Using outputs directly from mining-aster.ipynb and mining-palsar.ipynb")
    print("  No export/import needed - variables shared in same kernel session!")
    
    # Create integrated targets using existing variables
    integrated_targets = create_integrated_targets(
        structural_features,
        lineament_density,
        mineralization_probability
    )
    
    print("✓ Integrated analysis completed using existing outputs")
    
elif mineralization_loaded:
    print("⚠️  Using simplified structural features (full PALSAR analysis not available)")
    print("   For best results, run mining-palsar.ipynb first (all cells)")
    print("   Then run this notebook in the same kernel session")
    
    # Simplified structural features (fallback)
    if 'srtm' not in globals():
        srtm = ee.Image("USGS/SRTMGL1_003").clip(study_area_buffered)
    slope = ee.Terrain.slope(srtm)
    aspect = ee.Terrain.aspect(srtm)
    
    lineament_density_simple = slope.gt(10).reduceNeighborhood(
        reducer=ee.Reducer.count(),
        kernel=ee.Kernel.circle(radius=5, units='pixels')
    ).multiply(0.1).rename('Lineament_Density')
    
    faults_simple = slope.gt(15).And(aspect.gt(0)).rename('Faults')
    fractures_simple = slope.gt(10).And(slope.lte(15)).rename('Fractures')
    complexity_simple = lineament_density_simple.gt(3).rename('Structural_Complexity')
    shear_simple = lineament_density_simple.gt(2).And(lineament_density_simple.lte(3)).rename('Shear_Zones')
    dykes_simple = slope.gt(20).rename('Dykes')
    
    structural_features_simple = ee.Image.cat([
        faults_simple, fractures_simple, complexity_simple, 
        shear_simple, dykes_simple
    ])
    
    integrated_targets = create_integrated_targets(
        structural_features_simple,
        lineament_density_simple,
        mineralization_probability
    )
    
    print("✓ Integrated analysis completed (using simplified structural features)")
else:
    print("⚠️  Cannot perform integrated analysis - missing required inputs")
    print("   Please run mining-aster.ipynb and mining-palsar.ipynb first")
    print("   Then run this notebook in the same kernel session")
    integrated_targets = None

# Visualize integrated targets
if integrated_targets is not None:
    Map.addLayer(mineralization_probability, {
        'min': 0,
        'max': 1,
        'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
    }, 'Mineralization Probability', False)
    
    Map.addLayer(integrated_targets.select('Exploration_Score'), {
        'min': 0,
        'max': 1,
        'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
    }, 'Exploration Target Score (0-1)')
    
    Map.addLayer(integrated_targets.select('High_Priority'), {
        'min': 0,
        'max': 1,
        'palette': ['transparent', 'red']
    }, 'High Priority Targets (>0.7)', False)
    
    Map.addLayer(integrated_targets.select('Medium_Priority'), {
        'min': 0,
        'max': 1,
        'palette': ['transparent', 'orange']
    }, 'Medium Priority Targets (0.5-0.7)', False)
    
    Map.addLayer(integrated_targets.select('Low_Priority'), {
        'min': 0,
        'max': 1,
        'palette': ['transparent', 'yellow']
    }, 'Low Priority Targets (0.3-0.5)', False)
    
    Map.addLayer(integrated_targets.select('Priority_Targets'), {
        'min': 0,
        'max': 1,
        'palette': ['transparent', 'magenta']
    }, 'Priority: High Density + High Mineralization', False)
    
    Map.addLayer(integrated_targets.select('Structural_Control'), {
        'min': 0,
        'max': 1,
        'palette': ['transparent', 'cyan']
    }, 'Structural Control: Faults + Mineralization', False)
    
    Map.addLayer(integrated_targets.select('Favorable_Zones'), {
        'min': 0,
        'max': 1,
        'palette': ['transparent', 'green']
    }, 'Favorable Zones: Density + Mineralization', False)
    
    print("\n✓ Visualization layers added to map")
    print("\nDeliverables Created:")
    print("  1. Exploration Target Score: Weighted combination (0-1)")
    print("  2. High Priority Targets: Score >0.7 (red)")
    print("  3. Medium Priority Targets: Score 0.5-0.7 (orange)")
    print("  4. Low Priority Targets: Score 0.3-0.5 (yellow)")
    print("  5. Priority Targets: High density + High mineralization (magenta)")
    print("  6. Structural Control: Faults intersecting mineralization (cyan)")
    print("  7. Favorable Zones: Medium-high density + mineralization (green)")

print("=" * 60)
Map


In [ ]:
# Export Integrated Targets
# Export all integrated analysis deliverables

def export_integrated_targets():
    """Export integrated exploration target maps"""
    
    print("=" * 60)
    print("EXPORTING INTEGRATED EXPLORATION TARGETS")
    print("=" * 60)
    print("Exporting to Google Drive...")
    print("=" * 60)
    
    if 'integrated_targets' in globals() and integrated_targets is not None:
        geemap.ee_export_image(
            integrated_targets,
            filename='integrated_exploration_targets.tif',
            scale=30,
            region=study_area_buffered,
            file_per_band=False
        )
        print("✓ integrated_exploration_targets.tif - Integrated structural + mineralization")
        print("  Bands: Priority_Targets, Structural_Control, Favorable_Zones,")
        print("         Exploration_Score, High_Priority, Medium_Priority, Low_Priority")
    else:
        print("⚠️  No integrated targets to export. Run Cell 8 first.")
    
    if mineralization_loaded:
        geemap.ee_export_image(
            mineralization_probability,
            filename='mineralization_probability_2023_2024.tif',
            scale=30,
            region=study_area_buffered,
            file_per_band=False
        )
        print("✓ mineralization_probability_2023_2024.tif - Mineralization probability map")
    
    print("\n" + "=" * 60)
    print("EXPORT SUMMARY")
    print("=" * 60)
    print("All deliverables exported to Google Drive")
    print("Files ready for GIS analysis and interpretation")
    print("=" * 60)

print("Export function defined. Uncomment and call to export maps.")
print("Example: export_integrated_targets()")
